In [2]:
from datetime import datetime, timedelta
from collections import defaultdict

def get_month_range(target_month: str):
    start = datetime.strptime(target_month, "%Y-%m")
    next_month = start.replace(day=28) + timedelta(days=4)  # go to next month
    end = (next_month - timedelta(days=next_month.day)).replace(hour=23, minute=59, second=59)
    return start, end

def date_intersection(start1, end1, start2, end2):
    latest_start = max(start1, start2)
    earliest_end = min(end1, end2)
    if latest_start > earliest_end:
        return None
    return latest_start, earliest_end

def calculate_amount(rate, qty, start, end, active_start, active_end):
    total_days = (end - start).days + 1
    active_days = (active_end - active_start).days + 1
    daily_rate = (rate * qty) / total_days
    return round(daily_rate * active_days, 2)

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    target_start, target_end = get_month_range(target_month)
    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0, "billing_period": ""})

    for item in item_list:
        try:
            start = datetime.strptime(item["start_date"], "%Y-%m-%d")
            end = datetime.strptime(item["stop_date"], "%Y-%m-%d")
        except Exception:
            continue

        intersect = date_intersection(start, end, target_start, target_end)
        if not intersect:
            continue  # no overlap with target month

        active_start, active_end = intersect

        item_code = item.get("item_code")
        rate = float(item.get("rate", 0))
        qty = int(item.get("qty", 0))

        amount = calculate_amount(rate, qty, target_start, target_end, active_start, active_end)

        key = (item_code, rate, active_start.strftime("%Y-%m-%d") + " to " + active_end.strftime("%Y-%m-%d"))

        grouped_items[key]["qty"] += qty
        grouped_items[key]["amount"] += amount
        grouped_items[key]["billing_period"] = key[2]
        grouped_items[key]["item_code"] = item_code
        grouped_items[key]["rate"] = rate

    line_items = []
    total_revenue = 0.0

    for group in grouped_items.values():
        group["amount"] = round(group["amount"], 2)
        total_revenue += group["amount"]
        line_items.append(group)

    total_revenue = round(total_revenue, 2)

    return {
        "line_items": line_items,
        "total_revenue": total_revenue
    }

# Example usage:
item_list = [ 
    { "idx": 1, "item_code": "Executive Desk (4*2)", "sales_description": "Dedicated Executive Desk", "qty": 10, "rate": "1000", "amount": "10000", "start_date": "2023-11-01", "stop_date": "2024-10-17" }, 
    { "idx": 2, "item_code": "Executive Desk (4*2)", "qty": "10", "rate": "1080", "amount": "10800", "start_date": "2024-10-18", "stop_date": "2025-10-31" }, 
    { "idx": 3, "item_code": "Executive Desk (4*2)", "qty": 15, "rate": "1080", "amount": "16200", "start_date": "2024-11-01", "stop_date": "2025-10-31" }, 
    { "idx": 4, "item_code": "Executive Desk (4*2)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-01", "stop_date": "2025-10-31" }, 
    { "idx": 5, "item_code": "Manager Cabin", "qty": 5, "rate": 5000, "amount": 25000, "start_date": "2024-11-01", "stop_date": "2025-10-31" }, 
    { "idx": 6, "item_code": "Manager Cabin", "qty": 7, "rate": "5000", "amount": 35000, "start_date": "2024-12-15", "stop_date": "2025-10-31" }, 
    { "idx": 7, "item_code": "Manager Cabin", "qty": 10, "rate": 4600, "amount": 46000, "start_date": "2023-11-01", "stop_date": "2024-10-17" }, 
    { "idx": 8, "item_code": "Parking (2S)", "qty": 10, "rate": 1000, "amount": 10000, "start_date": "2024-11-01", "stop_date": "2025-10-31" }, 
    { "idx": 9, "item_code": "Parking (2S)", "qty": 10, "rate": 0, "amount": 0, "start_date": "2024-11-01", "stop_date": "2025-10-31" }, 
    { "idx": 10, "item_code": "Executive Desk (4*2)", "qty": "8", "rate": "1100", "amount": "8800", "start_date": "2024-11-15", "stop_date": "2025-01-31" }, 
    { "idx": 11, "item_code": "Manager Cabin", "qty": "3", "rate": "5200", "amount": "15600", "start_date": "2024-10-10", "stop_date": "2024-11-10" }, 
    { "idx": 12, "item_code": "Conference Table", "qty": 1, "rate": "20000", "amount": "20000", "start_date": "2024-11-05", "stop_date": "2024-11-20" }, 
    { "idx": 13, "item_code": "Parking (2S)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-15", "stop_date": "2025-02-28" }, 
    { "idx": 14, "item_code": "Reception Desk", "qty": 2, "rate": "7000", "amount": "14000", "start_date": "2024-11-01", "stop_date": "2025-03-31" }, 
    { "idx": 15, "item_code": "Reception Desk", "qty": 1, "rate": "7000", "amount": "7000", "start_date": "2024-11-10", "stop_date": "2024-11-25" }, 
    { "idx": 16, "item_code": "Breakout Area", "qty": 3, "rate": "3000", "amount": "9000", "start_date": "2024-01-01", "stop_date": "2024-01-31" } 
]

# Run the function for November 2024
result = generate_monthly_bill(item_list, "2024-11")

# Pretty print result
import json
print(json.dumps(result, indent=2))


{
  "line_items": [
    {
      "qty": 25,
      "amount": 27000.0,
      "billing_period": "2024-11-01 to 2024-11-30",
      "item_code": "Executive Desk (4*2)",
      "rate": 1080.0
    },
    {
      "qty": 5,
      "amount": 5000.0,
      "billing_period": "2024-11-01 to 2024-11-30",
      "item_code": "Executive Desk (4*2)",
      "rate": 1000.0
    },
    {
      "qty": 5,
      "amount": 25000.0,
      "billing_period": "2024-11-01 to 2024-11-30",
      "item_code": "Manager Cabin",
      "rate": 5000.0
    },
    {
      "qty": 10,
      "amount": 10000.0,
      "billing_period": "2024-11-01 to 2024-11-30",
      "item_code": "Parking (2S)",
      "rate": 1000.0
    },
    {
      "qty": 10,
      "amount": 0.0,
      "billing_period": "2024-11-01 to 2024-11-30",
      "item_code": "Parking (2S)",
      "rate": 0.0
    },
    {
      "qty": 8,
      "amount": 4693.33,
      "billing_period": "2024-11-15 to 2024-11-30",
      "item_code": "Executive Desk (4*2)",
      "rate": 11